In [1]:
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Activation
import numpy as np
import pandas as pd
from keras import initializers

import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import load_model
from keras import optimizers
from keras.callbacks import EarlyStopping
from tqdm import tqdm

Using TensorFlow backend.


# architecture

In [2]:
# construct the model used for classifier
# In our case the input shape is (42,1) is the board flattened

def classifier():  
    
    model = Sequential()
    # 2 "convo+pooling" layers
    model.add(Conv2D(filters=5, kernel_size=(3,3), strides=1, input_shape=(6,7,1)))
    #model.add(MaxPooling2D(pool_size=(2, 2),strides=2))
    model.add(Activation('relu'))
    
    model.add(Conv2D(filters=10, kernel_size=(3,3), strides=1))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=2))
    model.add(Activation('relu'))

    #flatten & linear layers
    model.add(Flatten())
    #model.add(Dense(42))
    model.add(Activation('relu'))

    #Classify if there is a merger waveform, output probabilistic
    model.add(Dense(2, activation="softmax"))

    return model

# compile

In [3]:
# Build model
model = classifier()

# compile model
ad = optimizers.Adam(lr=0.00005)
model.compile(loss='categorical_crossentropy', optimizer=ad, metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


# training data

In [4]:
# return a 6*7 array of zeros, which represent 
def newgame():
    n = np.zeros(42)
    n = np.reshape(n, (6,7))
    return n

In [5]:
# Check if col th column of B is full

def colAvailable(B, col):
    # start with checking if bottom (row 6) of ith column is empty 
    row = 5
    while B[row, col] != 0:
        row -= 1
        if row < 0:
            return False #column not available cuz full
    return True

In [6]:
# assume col not a full column, meaning it has been run through the columnfull function
# play a piece in ith column, [0,6] of B
# player denoted with -1, 1

# The function avoids modifying the original object numpy array B, returns a new object

def move(B, col, player):
    A = np.array(B)
    # start with checking if bottom (row 6) of ith column is empty 
    row = 5
    while A[row, col] != 0:
        row -= 1
    A[row, col] = player
    return A

In [45]:
# return all possible next board position, 
# and the associated probability of the current player winning based on the current model of network

def possibilities(B, player, network):
    pos = []
    for col in range(6):
        if colAvailable(B, col):
            moved0 = moved(B, col, player)
            moved = moved.reshape(6,7,1)
            moved = np.asarray([moved])
            pos.append([moved0, model.predict(moved)] )
    return pos

In [46]:
a = newgame()

                                                                           = move(a,-1)

In [38]:
b = possibilities(a,1,model)
print(len(b))

UnboundLocalError: local variable 'moved' referenced before assignment

In [34]:
b[0][0]

array([[[[ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.]],

        [[ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.]],

        [[ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.]],

        [[ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [-1.],
         [ 0.]],

        [[ 1.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [-1.],
         [ 0.]],

        [[ 1.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 1.],
         [ 0.]]]])

In [44]:
a

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -1., -1.]])

In [8]:
#Written by Ryan

def goodMove(B, network, player):
    poss = possibilities(B, network, player)
    
    if(player == 1):
        myIndex = 0
    else:
        myIndex=1
    
    total = 0
    u = np.random.uniform(0,1)
    
    for p in poss:
        total += p[1][0][myIndex]**3
        
    for i in range(len(poss)):
        sumCubeBefore=0
        for j in range(i):
            sumCubeBefore += poss[j][1][0][myIndex]**3
            
            
        #print("SumCubeBefore: " + str(sumCubeBefore))
        #print("u: " + str(u))
        #print("myIndex: " + str(myIndex))
        #print("poss[i][1]" + str(poss[i][1]))
        #print("sumCubeBefore + poss[i][1][myIndex]**3/total: " + str((sumCubeBefore + poss[i][1][0][myIndex]**3))
        #print("poss[i][1][myIndex]**3: " + str(poss[i][1][myIndex]**3))
        #print("poss[i][1]**3: " + str(poss[i][1][myIndex]**3) )
        print(u>=sumCubeBefore and u<=sumCubeBefore + poss[i][1][0][myIndex]**3) 
            
            
        if u>=sumCubeBefore/total and u<=(sumCubeBefore + poss[i][1][0][myIndex]**3)/total:
            A=poss[i][0][0]
    print()
    print()
    print()
    print()
    print()   
    print("this is A:" + str(A))
    print()
    print()
    print()
    print()
    print()
    return A
    

In [9]:
# board is the current position, a 2d numpy array 
# checkwin checks if any side wins
def checkwin(board):   #returns 1 if four 1s in a row, -1 if four -1s in a row, 0 otherwise
    print("This is board before" + str(board))
    #board = board[0]
    #print("This is board after" + str(board))
    
    isWin = 0

    wincombopos = [1,1,1,1]
    wincomboneg = [-1,-1,-1,-1]

    for row in board: #look for 4 of a kind on rows
        for i in range(4):
            if (np.array_equal(wincombopos, row[i:i+4])):
                isWin = 1
            elif (np.array_equal(wincomboneg, row[i:i+4])):
                isWin = -1
            
    for column in np.transpose(board): #look for 4 of a kind on columns
        for i in range(3):
            if (np.array_equal(wincombopos, column[i:i+4])):
                isWin = 1
            elif (np.array_equal(wincomboneg, column[i:i+4])):
                isWin = -1
    
    for ri in range(3): #row index   #look for 4 of a kind on downward sloping diagonals
        for ci in range(4): #column index
            diag = np.array([board[ri][ci], board[ri+1][ci+1], board[ri+2][ci+2], board[ri+3][ci+3]])
            if (np.array_equal(wincombopos, diag)):
                isWin = 1
            elif (np.array_equal(wincomboneg, diag)):
                isWin = -1
            
    for ri in range(3):
        for ci in range(4):
            diag = np.array([board[ri+3][ci], board[ri+2][ci+1], board[ri+1][ci+2], board[ri][ci+3]])
            if (np.array_equal(wincombopos, diag)):
                isWin = 1
            if (np.array_equal(wincomboneg, diag)):
                isWin = -1

    return isWin
    

In [10]:
# player1 goes first

def game(network, player1):
    B = newgame()
    step = 0
    boards = []
    probs = []
    
    if player1 == 1:
        player2 = -1
    else:
        player2 = 1
    
    # keep playing as long as no one winning
    while checkwin(B) == 0:
        step += 1
        if (step % 2) == 0: # odd step player1 moves, otherwise player2 moves
            B = goodMove(B, player2, network)
        else:
            B = goodMove(B, player1, network)
        boards.append(B)
    
    # try linear function first
    prob = np.arange(0.5, 1, 0.5/step) + 0.5/step
    
    X = boards
    Y = np.concatenate((prob,1-prob)).reshape(2,step)
        
    return X, Y


In [11]:
# number of training data to generate 

def generateData(count, network, player1):
    X = np.array([])
    Y = np.array([])
    for i in range(count):
        g = game(network,player1)
        X = np.concatenate((X,g[0]))
        Y = np.concatenate((Y,g[1]))
        
    return X, Y



In [12]:
game(model, 1)

This is board before[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]





this is A:[[[0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]]

 [[1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]]]





This is board before[[[0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]]

 [[1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]]]





this is A:[[[ 0.]
  [ 0.]
  [ 0.]
  [ 0.]
  [ 0.]
  [ 0.]
  [ 0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  [ 0.]
  [ 0.]
  [ 0.]
  [ 0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  [ 0.]
  [ 0.]
  [ 0.]
  [ 

UnboundLocalError: local variable 'A' referenced before assignment